In [127]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')
import pickle
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import numpy as np
import math
from data_preprocessing.session_traces_and_mean import get_all_experimental_records
from utils.reaction_time_utils import plot_reaction_times, plot_reaction_times_overlayed, get_valid_trials
from utils.correlation_utils import plot_all_valid_trials_over_time, plot_binned_valid_trials, multi_animal_scatter_and_fit
from utils.mean_trace_utils import mouseDates
from utils.post_processing_utils import remove_exps_after_manipulations, remove_bad_recordings
import data_preprocessing.bpod_data_processing as bpod
import pandas as pd
import os
import scipy as scipy
from scipy import optimize
from upgrade_plotting.across_session_changes import remove_experiments
from utils.plotting import calculate_error_bars

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
mouse = 'SNL_photo26'
recording_site = 'tail'
date = 'all'
all_experiments = get_all_experimental_records()
all_experiments = remove_exps_after_manipulations(all_experiments, [mouse])
all_experiments = remove_bad_recordings(all_experiments)
experiments_to_process = all_experiments[(all_experiments['mouse_id'] == mouse) & (all_experiments['recording_site'] == recording_site)]
dates = experiments_to_process['date'].values[0:2]
plot_reaction_times_overlayed(mouse, dates)

removing SNL_photo26: ['20200829' '20200830' '20200831' '20200908' '20200910' '20200911'
 '20200915' '20200917' '20200918' '20200921' '20201007' '20201008'
 '20201009']
Int64Index([52, 93, 208], dtype='int64')


In [307]:
mouse = 'SNL_photo26'
recording_site = 'tail'
date = '20200831'

In [309]:

saving_folder = 'W:\\photometry_2AC\\processed_data\\' + mouse + '\\'
num_types = len(dates)
colours = cm.viridis(np.linspace(0, 0.8, num_types))
mean_and_sem_filename = saving_folder +  mouse + '_' + date + '_' + 'aligned_traces.p'
data = pickle.load( open(mean_and_sem_filename, "rb" ))


In [310]:
recording_data = data.choice_data.ipsi_data
reaction_times = recording_data.reaction_times
traces = recording_data.sorted_traces
peaks = recording_data.trial_peaks
bins = np.arange(start=min(reaction_times), stop=max(reaction_times)+0.1, step=0.1)


In [311]:
new_peaks = [p for p in peaks if p.size>0]
ind = [i for i, p in enumerate(peaks) if p.size>0]

In [312]:
traces = recording_data.sorted_traces

In [313]:
trial_nums = recording_data.trial_nums

In [294]:
def linear(x, m, c):
    return ((m * x) + c)

In [295]:
def linear_fit(x_vals, y_vals):
    fit_equation = linear
    starting_params = [-0.0003, 0.5]
    legend = "y= %0.5f$x$ + %0.5f"
    popt_exponential, pcov_exponential = scipy.optimize.curve_fit(fit_equation, x_vals, y_vals, p0=starting_params)
    perr_exponential = np.sqrt(np.diag(pcov_exponential))
    x_vals_fit = np.linspace(np.min(x_vals), np.max(x_vals), 1000)
    y_vals_fit = fit_equation(x_vals_fit, popt_exponential[0], popt_exponential[1])
    residuals = y_vals - fit_equation(np.array(x_vals), popt_exponential[0], popt_exponential[1])
    ss_res = np.sum(residuals ** 2)
    ss_tot = np.sum((y_vals - np.mean(y_vals)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    print('r-squared value: ', r_squared)
    return x_vals_fit, y_vals_fit

In [296]:
x_vals_fit, y_vals_fit = linear_fit(reaction_times[ind], new_peaks)

r-squared value:  0.0065908101341849434


In [297]:
plt.figure()
plt.scatter(reaction_times[ind], np.array(new_peaks))
plt.plot(x_vals_fit, y_vals_fit, color='grey')

In [298]:
x_vals_fit, y_vals_fit = linear_fit(trial_nums[ind], new_peaks)

r-squared value:  0.0005591793735572059


In [299]:
plt.figure()
plt.scatter(trial_nums[ind], np.array(new_peaks))
plt.plot(x_vals_fit, y_vals_fit, color='grey')

In [314]:
reaction_times = np.abs(reaction_times)

In [315]:
upper_quartile = np.quantile(reaction_times, 0.75)
median = np.quantile(reaction_times, 0.5)
lower_quartile = np.quantile(reaction_times, 0.25)

In [316]:
long_reaction_times = np.where(reaction_times >=upper_quartile)[0]
middle_reaction_times = np.where(np.logical_and(reaction_times < upper_quartile, reaction_times >= median))[0]
short_reaction_times = np.where(np.logical_and(reaction_times < median, reaction_times >= lower_quartile))[0]
shortest_reaction_times =  np.where(reaction_times < lower_quartile)[0]

In [303]:
def get_traces_and_mean(all_traces, inds):
    traces = all_traces[inds, :]
    mean_trace = np.mean(traces, axis=0)
    return traces, mean_trace

In [304]:
def plot_trace(all_traces, inds, time_stamps, ax, color='green'):
    traces, mean_trace = get_traces_and_mean(all_traces, inds)

    ax.plot(time_stamps, mean_trace, color=color)
    error_bar_lower, error_bar_upper = calculate_error_bars(mean_trace,
                                                            traces,
                                                            error_bar_method='sem')
    ax.fill_between(time_stamps, error_bar_lower, error_bar_upper, alpha=0.5,
                     facecolor=color, linewidth=0)
    ax.set_xlabel('time(s)')
    ax.set_ylabel('z-scored fluorescence')


In [317]:
time_stamps= recording_data.time_points

In [318]:
fig, axs = plt.subplots(1,1, sharey=True)
plot_trace(traces, long_reaction_times, time_stamps, axs, color='blue')
plot_trace(traces, middle_reaction_times, time_stamps, axs, color='orange')
plot_trace(traces, short_reaction_times, time_stamps, axs, color='green')
plot_trace(traces, shortest_reaction_times, time_stamps, axs, color='red')

In [320]:
reaction_times.shape

(263,)